In [1]:
#############################################################################################
#Как работает данная модель сверточной нейронной сети: 
#    1. По адресу: https://drive.google.com/drive/folders/1x5-92pU8kzd-UngaddUCR7WJtDrbAG1N?usp=drive_link - расположены .wav-файлы со звуком плохой фистулы  (для работы модели сохраняла файлы локально в папку "../fistulas_sound/bad", всего 6 аудио-файлов)
#    2. По адресу: https://drive.google.com/drive/folders/1EmN3jzet_dXdnmFG8wu0KpShHi84tKMM?usp=drive_link - расположены .wav-файлы со звуком хорошей фистулы (для работы модели сохраняла файлы локально в папку "../fistulas_sound/good", всего 4 аудио-файла)
#    3. По адресу: https://drive.google.com/file/d/1mk6ppeIqiBpDIMBebteRo8jqQDU0WB9I/view?usp=drive_link - расположены заархивированные .wav-файлы, подлежащие классификации (для работы модели сохраняла файлы локально в папку "../fistulas_sound/vita-fs-data", всего 875 аудио-файлов)
#    4. С помощью библиотеки librosa тренировочные аудио-файлы (п.1-2) из формата .wav перегоняются в формат .png (спектрограммы) и сохраняются в папки "../img_data/good" (4 спектрограммы) и "../img_data/bad" (6 спектрограмм) соответственно.
#    5. Разбиваем исходный, классифицированный ранее, набор данных (п.1-2) на обучающую и тестовую выборки в соотношении 80/20: для этого сохраняем в папки "../data/train/bad" (четыре "плохих" спектрограммы из шести) и "../data/train/good" (три "хороших" спектрограммы из четырёх) 80% исходного набора тренировочных данных, а в папки "../data/val/bad" (две оставшиеся "плохих" спектрограммы из шести) и "../data/val/good" (одна оставшаяся "хорошая" спектрограмма из четырёх) - 20% исходного набора тренировочных данных.
#    6. Используя класс Sequential() из keras формируем модель свёрточной нейронной сети (convolutional neural network (CNN)).
#    7. Обучаем модель с помощью стохастического градиентного спуска (SGD).
#    8. Производим оценку модели, результаты такие: потери - 0,66%, точность - 67%. Для большей точности нужно оптимизировать модель (поле для разработки!).
#    9. Выгружаем модель свёрточной нейронной сети в формат .pickle, подходящий для загрузки в ИС "Vita-Control", позволяющей выполнять бинарную классификацию данных через пользовательский интерфейс (загрузка в БД ИС - отдельная задача). 
#    10. Переносим максимальное кол-во файлов, кратное размеру пачки (в данном случае - 32) в формат .png действиями, аналогичными действиям из п.4. Из доступных 875 файлов классификации будет подвержено 864 файла (875 - (875 mod 32) = 864). Спектрограммы сохраняем в папку "../data/test/unclassified".   
#    11. Выполняем прогноз на основном тестовом наборе данных (864 спектрограммы звука из папки "../data/test/unclassified").
#    12. Сопоставляем результаты классификации (результаты не нравятся - здесь проверить код! попозже посмотрю)
#    13. Записываем результат бинарной классификации в файл "prediction_results_CNN_model.csv".
#P.S. Источник - https://nuancesprog.ru/p/6758/ - всё очень понятно написано. 
#############################################################################################

In [2]:
#п.1-3 делаем до запуска IDE (формировала модель в локальном jupyter-notebook.
#Далее в листинге в комментах будет указан номер пункта из описанной выше работы модели.

In [3]:
#импортируем нужные библиотеки
import pandas as pd
import numpy as np
from numpy import argmax
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import librosa.display
import IPython.display
import random
import warnings
import os
from PIL import Image
import pathlib
import csv
from sklearn.model_selection import train_test_split

In [4]:
#подключаем tensorflow, keras - в его составе
import tensorflow as tf
from tensorflow import keras
import warnings
warnings.filterwarnings('ignore')

In [5]:
#подключаем элементы keras
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from keras.models import Sequential
from keras.optimizers import SGD #градиентный спуск

In [6]:
#п.4 описания - формируем директорию img_data, содержащую все изображения, классифицированные по видам фистул: плохая/хорошая
fistulas_kind = 'good bad'.split()
for f in fistulas_kind:
    pathlib.Path(f'img_data/{f}').mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(f'fistulas_sound/{f}/'):
        wav_name = f'fistulas_sound/{f}/{filename}'
        y, sr = librosa.load(wav_name, mono=True, duration=5)
        #print(y.shape)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap='Blues', sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{f}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 640x480 with 0 Axes>

In [7]:
import splitfolders
#п.5 описания - разбиваем на обучающую и тестовую выборки: 80/20
splitfolders.ratio('./img_data/', output="./data", seed=1337, ratio=(.8, .2)) # значения по умолчанию

Copying files: 10 files [00:00, 76.46 files/s]


In [8]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255, #изменение масштаба всех значений пикселей с 0 до 255, после этого шага они будут находится в диапазоне (0..1)
        shear_range=0.2, #применение случайных преобразований
        zoom_range=0.2, #увеличение масштаба
        horizontal_flip=True) #горизонтальный поворот
test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
training_set = train_datagen.flow_from_directory(
        './data/train',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical',
        shuffle = False)
test_set = test_datagen.flow_from_directory(
        './data/val',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical',
        shuffle = False )

Found 7 images belonging to 2 classes.
Found 3 images belonging to 2 classes.


In [10]:
#п.6 описания - формируем модель свёрточной нейронной сети:
model_CNN = Sequential()
input_shape=(64, 64, 3)

#первый скрытый слой
model_CNN.add(Conv2D(32, (3, 3), strides=(2, 2), input_shape=input_shape))
model_CNN.add(AveragePooling2D((2, 2), strides=(2,2)))
model_CNN.add(Activation('relu'))

#второй скрытый слой
model_CNN.add(Conv2D(64, (3, 3), padding="same"))
model_CNN.add(AveragePooling2D((2, 2), strides=(2,2)))
model_CNN.add(Activation('relu'))

#третий скрытый слой
model_CNN.add(Conv2D(64, (3, 3), padding="same"))
model_CNN.add(AveragePooling2D((2, 2), strides=(2,2)))
model_CNN.add(Activation('relu'))

#слой выравнивания
model_CNN.add(Flatten())
model_CNN.add(Dropout(rate=0.5))

#полносвязный слой
model_CNN.add(Dense(64))
model_CNN.add(Activation('relu'))
model_CNN.add(Dropout(rate=0.5))

#выходной слой
model_CNN.add(Dense(2))
model_CNN.add(Activation('softmax'))
model_CNN.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 31, 31, 32)        896       
                                                                 
 average_pooling2d (Average  (None, 15, 15, 32)        0         
 Pooling2D)                                                      
                                                                 
 activation (Activation)     (None, 15, 15, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 15, 15, 64)        18496     
                                                                 
 average_pooling2d_1 (Avera  (None, 7, 7, 64)          0         
 gePooling2D)                                                    
                                                                 
 activation_1 (Activation)   (None, 7, 7, 64)          

In [11]:
#п.7 описания - обучаем модель - команда compile
epochs = 10
batch_size = 8
learning_rate = 0.01
decay_rate = learning_rate/epochs
momentum = 0.9
sgd = SGD(learning_rate=learning_rate, momentum=momentum, weight_decay=decay_rate, nesterov=False) #sgd - это стох.град.спуск
model_CNN.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=['accuracy']) #кросс-энтропию - проверить, возможно, заменить параметры модели

In [12]:
model_CNN.fit_generator(
        training_set,
        steps_per_epoch=1,
        epochs=10,
        validation_data=test_set,
        validation_steps=1)

Epoch 1/10


1/1 [==============================] - 2s 2s/step - loss: 0.7895 - accuracy: 0.4286 - val_loss: 0.6728 - val_accuracy: 0.6667
Epoch 2/10
1/1 [==============================] - 0s 211ms/step - loss: 0.6771 - accuracy: 0.7143 - val_loss: 0.6774 - val_accuracy: 0.6667
Epoch 3/10
1/1 [==============================] - 0s 206ms/step - loss: 0.7497 - accuracy: 0.4286 - val_loss: 0.6827 - val_accuracy: 0.6667
Epoch 4/10
1/1 [==============================] - 0s 224ms/step - loss: 0.7010 - accuracy: 0.4286 - val_loss: 0.6882 - val_accuracy: 0.6667
Epoch 5/10
1/1 [==============================] - 0s 200ms/step - loss: 0.6715 - accuracy: 0.5714 - val_loss: 0.6807 - val_accuracy: 0.6667
Epoch 6/10
1/1 [==============================] - 0s 200ms/step - loss: 0.7719 - accuracy: 0.4286 - val_loss: 0.6821 - val_accuracy: 0.6667
Epoch 7/10
1/1 [==============================] - 0s 192ms/step - loss: 0.6960 - accuracy: 0.4286 - val_loss: 0.6724 - val_accuracy: 0.6667
Epoch 8/10
1/1 [=====

In [13]:
#п.8 описания - оценка модели
model_CNN.evaluate_generator(generator=test_set, steps=1) #потери - 0,66%, точность - 67%, мало :P

[0.6696901321411133, 0.6666666865348816]

In [14]:
#п.9 описания - выгружаем модель свёрточной нейронной сети в формат .pickle для загрузки в ИС "Vita-Control", позволяющей выполнять классификацию данных через GUI (Web). 
import pickle
data = model_CNN
with open('model_CNN.pickle', 'wb') as f:
  pickle.dump(data, f)

In [44]:
#п.10 описания - переносим основные данные wav->png для классификации с пом.построенной модели (переношу только 864 файла из 875, т.к. размер пачки установила (ниже) = 32),
#для вывода рез-тов классиф-ии в файл нужно, чтобы совпадало кол-во сэмплов и рез-тов (это 2 столбца в результирующем .csv)
li_cnt = 0 #счётчик перенесенных файлов
pathlib.Path(f'data/test/unclassified').mkdir(parents=True, exist_ok=True)
for filename in os.listdir(f'fistulas_sound/vita-fs-data/'):
    if li_cnt < 864:
        try:
            wav_name = f'fistulas_sound/vita-fs-data/{filename}'
            y, sr = librosa.load(wav_name, mono=True, duration=5)
            print(y.shape)
            plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap='Blues', sides='default', mode='default', scale='dB');
            plt.axis('off');
            plt.savefig(f'data/test/unclassified/{filename[:-3].replace(".", "")}.png')
            plt.clf()
            li_cnt = li_cnt + 1
        except:
            print(filename)
    else:
        break

(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)


(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
cp.sh
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(39514,)
(62093,)
(50804,)
(39514,)
(45159,)
(28224,)
(33869,)
(28224,)
(39514,)
(28224,)
(62093,)
(28224,)
(50804,)
(39514,)
(39514,)
(16935,)
(28224,)
(67738,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)


<Figure size 640x480 with 0 Axes>

In [15]:
main_test_set = test_datagen.flow_from_directory(
        './data/test',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical',
        shuffle = False )

Found 864 images belonging to 1 classes.


In [16]:
#п.11 описания - выполняем прогноз на основном тестовом наборе данных (864 спектрограммы звука).
main_test_set.reset()
pred = model_CNN.predict_generator(main_test_set, steps=27, verbose=1)

27/27 [==============================] - 33s 1s/step


In [17]:
#п.12 описания - сопоставляем рез-ты классификации
predicted_class_indices=np.argmax(pred,axis=1)
labels = (training_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
predictions = predictions[:864]
filenames=main_test_set.filenames

In [18]:
#защита от дурака - кол-во классифицируемых записей д.б. = кол-ву выполненных прогнозов.
print(len(filenames), len(predictions))

864 864


In [19]:
#п.13 описания - записываем рез-т в файл "prediction_results_CNN_model.csv"
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("prediction_results_CNN_model.csv",index=False)